<a href="https://colab.research.google.com/github/AnnaTravelblog/AnnaTravelblog/blob/main/mybot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install python-telegram-bot==13.7

In [7]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from telegram.ext import CallbackContext
import os
import random
import requests

In [ ]:
TOKEN = '6997691044:AAENTkgjfG1bHQ03IrcBiJNbeAAnQ0CmXvk'
OPENWEATHERMAP_API_KEY = 'f98028a3cd35a770ee7e30d447f45301'

BASE_URL = f'https://api.telegram.org/bot{TOKEN}/'
WEATHER_API_URL = 'https://api.openweathermap.org/data/2.5/weather?'

# Приветственные сообщения
GREETINGS = [
    "Привет! Я твой путешественник-бот. Готов помочь с планированием твоих следующих приключений!",
    "Здравствуй! Я готов отправиться с тобой в путешествие!",
    "Приветствую тебя, путешественник! Куда собираешься отправиться?",
    "Привет! Я здесь, чтобы помочь тебе с выбором следующего направления для путешествия."
]

DESTINATIONS = {
    "париж": "Париж - прекрасный город! Обязательно посети Эйфелеву башню и Лувр.",
    "токио": "Токио - удивительный город! Не пропусти Цукадзи и Сенсодзи.",
    "рим": "Рим - великолепный город! Посети Колизей и Ватикан.",

    "лондон": "Лондон - столица Великобритании! Посети Биг Бен и Британский музей.",

    "барселона": "Барселона - город архитектурных шедевров! Посети собор Святого Семейства и Парк Гуэль.",

    "нью-йорк":
        "Нью-Йорк - город, который никогда не спит! Открой для себя Манхэттен и Центральный парк.",

    "москва":
        "Москва - столица России! Посети Красную площадь и Кремль.",

    "киев":
         "Киев - древний город с богатой историей! Посети Софиевский собор и Подол.",

    "бангкок":
         "Бангкок - экзотический город в сердце Таиланда! Наслаждайся вкусной тайской кухней и посети храм Ват Арун.",

    "прага":
        "Прага - город старинных улочек и замков! Посети Карлов мост и Пражский град.",

    "дубай":
         "Дубай - город впечатляющих небоскребов и роскоши! Посети Бурдж Халифа и Дубай Молл.",

    "сидней":
         "Сидней - живописный город на побережье! Посети оперный театр и Сиднейский харбор.",

    "амстердам":
        "Амстердам - город каналов и великолепной архитектуры! Посети музей Ван Гога и Дом Анны Франк.",

    "флоренция":
        "Флоренция - исторический город с богатым культурным наследием! Посети Галерею Уффици и собор Санта-Мария-дель-Фьоре.",

    "киото":
         "Киото - древняя столица Японии с множеством храмов и садов! Посети Золотой павильон и храм Фушими Инари.",

    "сан-франциско":
         "Сан-Франциско - культурный и технологический центр США! Посети знаменитый мост Голден Гейт и остров Алькатрас.",

    "шанхай":
         "Шанхай - современный мегаполис с высокими небоскребами! Посети набережную Бунд и Ципао.",

    "кипр":
         "Кипр - остров любви и истории! Посети археологический парк Пафоса и монастырь Киккос.",

    "рио-де-жанейро":
         "Рио-де-Жанейро - город пляжей и карнавалов! Посети Христа-Искупителя и пляж Ипанема.",

    "сеул":
        "Сеул - космополитичный город с высокими технологиями! Посети дворец Гёнбоккун и Торговый центр Дунгдаемун.",

    "будапешт":
        "Будапешт - город с тысячелетней историей и термальными источниками! Посети Рыбацкий бастион и парламент.",

    "каир":
        "Каир - древний город с пирамидами и сфинксами! Посети пирамиду Хеопса и музей египетской античности.",

    "санкт-петербург":
        "Санкт-Петербург - северная столица России с богатым культурным наследием! Посети Эрмитаж и Петергоф.",

    "мельбурн":
         "Мельбурн - культурный и спортивный центр Австралии! Посети здание парламента и Royal Botanic Gardens.",

     "дублин":
         "Дублин - живописный город с историческими замками и пабами! Посети Замок Дублин и бар Темпл Бар.",

    "вена":
        "Вена - столица Австрии с красивой архитектурой и классической музыкой! Посети дворец Шенбрунн и кафе Централь.",

    "кишинев":
      "Кишинёв - столица Молдовы! Посети Национальный музей и Старую крепость.",

    "тирасполь":
        "Тирасполь - столица Приднестровской Молдавской Республики! Посети Приднестровский государственный университет и Парк Славы.",

    "кипр":
        "Кипр - остров любви и истории! Посети археологический парк Пафоса и монастырь Киккос.",

    "рио-де-жанейро":
       "Рио-де-Жанейро - город пляжей и карнавалов! Посети Христа-Искупителя и пляж Ипанема.",

    "сеул":
        "Сеул - космополитичный город с высокими технологиями! Посети дворец Гёнбоккун и Торговый центр Дунгдаемун.",

    "пекин":
        "Пекин - столица Китая с богатой историей! Посети Запретный город и Великую Китайскую стену.",

    "мумбай":
         "Мумбай - крупнейший город Индии! Посети Ворота Индии и Храм Махалакшми.",

    "рейкьявик":
         "Рейкьявик - столица Исландии! Посети Холлгримскерку и смотровую площадку Перлан.",

    "санторини":
     "Санторини - жемчужина Эгейского моря! Посети город Фира и пляж Камари.",

    "венеция":
         "Венеция - город на воде! Посети площадь Сан-Марко и остров Мурано."}

def handle_message(update: Update, context: CallbackContext) -> None:
    text = update.message.text.lower()
    destination = DESTINATIONS.get(text)
    if destination:
        weather_info = get_weather_info(text)
        send_message(update.message.chat_id, f"{destination}\n\n{weather_info}")
    elif "спасибо" in text:
        send_message(update.message.chat_id, "Пожалуйста!")
    elif "всегда пожалуйста" in text:
        send_message(update.message.chat_id, "Рад помочь!")
    else:
        send_message(update.message.chat_id, "Я не знаю такого места. Может быть, выбери что-то другое?")

def get_weather_info(city: str) -> str:
    params = {'q': city, 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric', 'lang': 'ru'}
    response = requests.get(WEATHER_API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        return f"Погода в {city.capitalize()}: {weather_description}, Температура: {temperature}°C"
    else:
        return "Информация о погоде недоступна."

def send_message(chat_id, text, reply_markup=None):
    data = {
        'chat_id': chat_id,
        'text': text,
        'reply_markup': reply_markup
    }
    response = requests.post(BASE_URL + 'sendMessage', data=data)
    print(response.json())

def start(update, context):
    update.message.reply_text(random.choice(GREETINGS) + " В какой город вы хотите поехать?")

def main():
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    # Обработчик команды /start
    dispatcher.add_handler(CommandHandler("start", start))

    # Добавление обработчика для обычных текстовых сообщений
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

{'ok': True, 'result': {'message_id': 11, 'from': {'id': 6997691044, 'is_bot': True, 'first_name': 'Aabot', 'username': 'Anna86Bot'}, 'chat': {'id': 1134454737, 'first_name': 'Анна', 'last_name': 'Руд', 'username': 'AnnaRudetckaia', 'type': 'private'}, 'date': 1712671060, 'text': 'Я не знаю такого места. Может быть, выбери что-то другое?'}}
{'ok': True, 'result': {'message_id': 16, 'from': {'id': 6997691044, 'is_bot': True, 'first_name': 'Aabot', 'username': 'Anna86Bot'}, 'chat': {'id': 1134454737, 'first_name': 'Анна', 'last_name': 'Руд', 'username': 'AnnaRudetckaia', 'type': 'private'}, 'date': 1712671105, 'text': 'Кишинёв - столица Молдовы! Посети Национальный музей и Старую крепость.\n\nПогода в Кишинев: ясно, Температура: 24.24°C'}}
{'ok': True, 'result': {'message_id': 17, 'from': {'id': 6997691044, 'is_bot': True, 'first_name': 'Aabot', 'username': 'Anna86Bot'}, 'chat': {'id': 1134454737, 'first_name': 'Анна', 'last_name': 'Руд', 'username': 'AnnaRudetckaia', 'type': 'private'},